In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.chdir(os.path.dirname(os.getcwd()))
import cvxpy as cp
import numpy as np
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.model_selection import train_test_split
from survLime.datasets.load_datasets import RandomSurvivalData
from survLime import survlime_tabular
from survLime.scripts.experiment_1 import create_clusters
from survLime.utils.generic_utils import compare_survival_times
np.random.seed(42)

In [ ]:
# Create clusters can be found in scripts/experiment_1
cluster_0, cluster_1 = create_clusters()
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(cluster_0[0],
                                                            cluster_0[1], test_size=0.1)
times = [x[1] for x in y_train_1]
times_to_fill = list(set(times))
times_to_fill.sort() 

In [ ]:
# Train bb model
columns =['one']
model = CoxPHSurvivalAnalysis(alpha=0.0001)
model.fit(x_train_1, y_train_1)
model.feature_names_in_ = columns # This is needed in order to compare survival times later

# Obtain a test point to use for prediction
test_point = x_test_1[0]

In [ ]:
# We instantiate an explainer (found in survlime_tabular)

# ['one','two',' three', 'four', 'five']

explainer = survlime_tabular.LimeTabularExplainer(x_train_1, y_train_1,
                                                  feature_names= columns,
                                                  class_names=None, 
                                                  categorical_features=None,
                                                  verbose=True, discretize_continuous=False)
num_neighbours = 1000
# From here we are only using log_correction, Ho_t_ and inverse 
H_i_j_wc, weights, log_correction, Ho_t_, scaled_data, inverse, distances = explainer.explain_instance(test_point
                                                                ,model.predict_cumulative_hazard_function,
                                                            num_samples = num_neighbours)

In [ ]:
scaled_data.shape

## From this cell onwards we construct the matrices
We need 
* H, 
* H0
* b
* weights 
* delta_t

In [ ]:
# Constants
epsilon = 0.000001
m = len(times_to_fill)

# Variable of interest
b = cp.Variable(shape=(1,1))

# Compute the H for all the neighbours
yss = model.predict_cumulative_hazard_function(scaled_data, return_array=True)

H = yss.copy()
LnH = np.log(H)

# Baseline cumulative hazard 
H0 = np.reshape(Ho_t_, (m,1))
LnH0 = np.log(H0)

# Compute the log correction
logs = np.reshape(log_correction, (num_neighbours, m))

# Distance weights
w = np.reshape(weights, (num_neighbours,1))

# Time differences
t = times_to_fill.copy()
t.append(t[-1]+1)
delta_t = [t[i+1] - t[i] for i in range(m)]
delta_t = np.reshape(np.array(delta_t), newshape=(m, 1))

In [ ]:
# Matrices to produce the propper sizes
ones_N = np.ones(shape=(num_neighbours, 1))
ones_m_1 = np.ones(shape=(m, 1))

B = np.dot(ones_N, LnH0.T)
C = LnH-B

Z = scaled_data@b
D = Z@ones_m_1.T

E = C - D
E_sq = cp.square(E)

F = E_sq@delta_t
funct = F.T@w
objective = cp.Minimize(funct)
prob = cp.Problem(objective)
result = prob.solve()

In [ ]:
print(funct.value)
print(b.value)
print(model.coef_)

In [ ]:
# Coefficient coppied from survLime/scripts/experiment_1.create_clusters
coefficients = [0.2]# [10**(-6), 0.1,  -0.15, 10**(-6), 10**(-6)] 
values = [x[0] for x in b.value]
compare_survival_times(model, values, x_train_1, y_train_1, x_test_1, true_coef=coefficients)